In [ ]:
!pip install git+https://github.com/spiceai/spicepy

In [ ]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import plotly.io as pio
pio.templates.default = "seaborn"

In [ ]:
from spicepy import Client

client = Client('323430|2a78fb24afde4a89bdacb3a7e00d2421') # Community API Key for Kaggle

# Get Data 
## UniswapV2
### USDC/WETH, WETH/USDT, USDC/USDT

In [ ]:
def get_specific_dex_pair(p0='USDC', p1='WETH', dex='uniswap_v2'):
    
    query = f"""
        SELECT
        AVG(reserve0) AS {p0}_reserve,
        AVG(reserve1) AS {p1}_reserve,
        AVG(price0) AS {p0}_{p1}_price,
        AVG(price1) AS {p1}_{p0}_price,
        DATE_TRUNC('minute', to_timestamp(block_timestamp_last)) AS "timestamp"
        FROM eth.{dex}.pool_stats_detailed
        WHERE token0_symbol = '{p0}' and token1_symbol = '{p1}'
        GROUP BY "timestamp", pool_address, token0_symbol, token1_symbol
        ORDER BY "timestamp" DESC
    """
    
    df = client.query(query).read_pandas()
    return df

In [ ]:
df_usdc_weth = get_specific_dex_pair(p0='USDC', p1='WETH')
df_usdc_weth.to_csv('usdc_weth_uniswapV2.csv')
# df_weth_usdt = get_specific_dex_pair(p0='WETH', p1='USDT')
# df_weth_usdt.to_csv('weth_usdt_uniswapV2.csv')
df_usdc_usdt = get_specific_dex_pair(p0='USDC', p1='USDT')
df_usdc_usdt.to_csv('usdc_usdt_uniswapV2.csv')

### Map DEX Pool address and pairs

In [ ]:
def map_dex_pool_pairs(dex='uniswap_v2'):
    
    query = f"""
        SELECT
        pool_address,
        token0_symbol,
        token1_symbol
        FROM eth.{dex}.pool_stats_detailed
        GROUP BY pool_address, token0_symbol, token1_symbol
    """
    
    df = client.query(query).read_pandas()
    return df

In [ ]:
df_map_pool_pairs = map_dex_pool_pairs(dex='uniswap_v2')
df_map_pool_pairs

In [ ]:
df_map_pool_pairs.to_csv('map_pool_pairs.csv')

In [ ]:
# 0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8 -> USDC/WETH
# 0x3416cf6c708da44db2624d63ea0aaef7113527c6 -> USDC/USDT

def get_pool_stats_UniswapV3(pool_address, dex="uniswap_v3"):
    query = f"""
        SELECT
        block_number,
        pool_address,
        liquidity_hex,
        max_liquidity_per_tick_hex,
        slot0_sqrt_price_x96_hex
        FROM eth.{dex}.pool_stats
        WHERE pool_address = '{pool_address}'
        ORDER BY "block_number" DESC
    """
    
    df = client.query(query).read_pandas()
    
    return df
    
df_pool_stats_usdc_weth_v3 = get_pool_stats_UniswapV3("0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8", dex="uniswap_v3")
df_pool_stats_usdc_weth_v3.to_csv('pool_stats_usdc_weth_uniswapV3.csv')
df_pool_stats_usdc_usdt_v3 = get_pool_stats_UniswapV3("0x3416cf6c708da44db2624d63ea0aaef7113527c6", dex="uniswap_v3")
df_pool_stats_usdc_usdt_v3.to_csv('pool_stats_usdc_usdt_uniswapV3.csv')

### Swaps

In [ ]:
def get_pair_swaps_uniswapV2(df_map_pool_pairs, p0, p1, dex='uniswap_v2'):
    df_map_pool_pair = df_map_pool_pairs[(df_map_pool_pairs['token0_symbol'] == p0) & (df_map_pool_pairs['token1_symbol'] == p1)]
    pool_address = df_map_pool_pair['pool_address'].values[0]
    query = f"""
        SELECT 
        address,
        amount0_in,
        amount1_in,
        amount0_out,
        amount1_out,
        sender,
        to_address,
        transaction_hash,
        DATE_TRUNC('minute', to_timestamp(block_timestamp)) AS "timestamp"
        FROM eth.{dex}.event_swaps
        WHERE address = '{pool_address}'
        ORDER BY "timestamp" DESC
    """
    
    df = client.query(query).read_pandas()
    
    return df

def get_pair_swaps_uniswapV3(pool_address, dex='uniswap_v3'):
    query = f"""
        SELECT 
        address,
        amount0,
        amount1,
        sqrt_price_x96,
        liquidity,
        tick,
        sender,
        recipient,
        transaction_hash,
        block_number,
        DATE_TRUNC('minute', to_timestamp(block_timestamp)) AS "timestamp"
        FROM eth.{dex}.event_swaps
        WHERE address = '{pool_address}'
        ORDER BY "timestamp" DESC
    """
    
    df = client.query(query).read_pandas()
    
    return df

In [ ]:
# df_swaps_usdc_weth = get_pair_swaps_uniswapV2(df_map_pool_pairs, p0='USDC', p1='WETH', dex='uniswap_v2')
# df_swaps_usdc_weth.to_csv('swaps_usdc_weth_uniswapV2.csv')
# df_swaps_weth_usdt = get_pair_swaps_uniswapV2(df_map_pool_pairs, p0='WETH', p1='USDT', dex='uniswap_v2')
# df_swaps_weth_usdt.to_csv('swaps_weth_usdt_uniswapV2.csv')
# df_swaps_usdc_usdt = get_pair_swaps_uniswapV2(df_map_pool_pairs, p0='USDC', p1='USDT', dex='uniswap_v2')
# df_swaps_usdc_usdt.to_csv('swaps_usdc_usdt_uniswapV2.csv')

# 0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8 -> USDC/WETH
# 0x3416cf6c708da44db2624d63ea0aaef7113527c6 -> USDC/USDT
df_swaps_usdc_weth_v3 = get_pair_swaps_uniswapV3("0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8", dex='uniswap_v3')
df_swaps_usdc_weth_v3.to_csv('swaps_usdc_weth_uniswapV3.csv')
df_swaps_usdc_usdt_v3 = get_pair_swaps_uniswapV3("0x3416cf6c708da44db2624d63ea0aaef7113527c6", dex='uniswap_v3')
df_swaps_usdc_usdt_v3.to_csv('swaps_usdc_usdt_uniswapV3.csv')

### Mints

In [ ]:
def get_pair_mints_uniswapV2(df_map_pool_pairs, p0, p1, dex='uniswap_v2'):
    df_map_pool_pair = df_map_pool_pairs[(df_map_pool_pairs['token0_symbol'] == p0) & (df_map_pool_pairs['token1_symbol'] == p1)]
    pool_address = df_map_pool_pair['pool_address'].values[0]
    
    query = f"""
        SELECT 
        address,
        amount0,
        amount1,
        sender,
        transaction_hash,
        DATE_TRUNC('minute', to_timestamp(block_timestamp)) AS "timestamp"
        FROM eth.{dex}.event_mints
        WHERE address = '{pool_address}'
        ORDER BY "timestamp" DESC
    """
    
    df = client.query(query).read_pandas()
    
    return df

def get_pair_mints_uniswapV3(pool_address, dex="uniswapv3"):
    query = f"""
        SELECT 
        address,
        tick_lower,
        tick_upper,
        amount,
        amount0,
        amount1,
        owner,
        sender,
        transaction_hash,
        block_number,
        DATE_TRUNC('minute', to_timestamp(block_timestamp)) AS "timestamp"
        FROM eth.{dex}.event_mints
        WHERE address = '{pool_address}'
        ORDER BY "timestamp" DESC
    """
    
    df = client.query(query).read_pandas()
    
    return df

In [ ]:
# df_mints_usdc_weth = get_pair_mints_uniswapV2(df_map_pool_pairs, p0='USDC', p1='WETH', dex='uniswap_v2')
# df_mints_usdc_weth.to_csv('mints_usdc_weth_uniswapV2.csv')
# df_mints_weth_usdt = get_pair_mints_uniswapV2(df_map_pool_pairs, p0='WETH', p1='USDT', dex='uniswap_v2')
# df_mints_weth_usdt.to_csv('mints_weth_usdt_uniswapV2.csv')
# df_mints_usdc_usdt = get_pair_mints_uniswapV2(df_map_pool_pairs, p0='USDC', p1='USDT', dex='uniswap_v2')
# df_mints_usdc_usdt.to_csv('mints_usdc_usdt_uniswapV2.csv')

# 0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8 -> USDC/WETH
# 0x3416cf6c708da44db2624d63ea0aaef7113527c6 -> USDC/USDT
df_mints_usdc_weth_v3 = get_pair_mints_uniswapV3("0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8", dex='uniswap_v3')
df_mints_usdc_weth_v3.to_csv('mints_usdc_weth_uniswapV3.csv')
df_mints_usdc_usdt_v3 = get_pair_mints_uniswapV3("0x3416cf6c708da44db2624d63ea0aaef7113527c6", dex='uniswap_v3')
df_mints_usdc_usdt_v3.to_csv('mints_usdc_usdt_uniswapV3.csv')

### Burns

In [ ]:
def get_pair_burns_uniswapV2(df_map_pool_pairs, p0, p1, dex='uniswap_v2'):
    df_map_pool_pair = df_map_pool_pairs[(df_map_pool_pairs['token0_symbol'] == p0) & (df_map_pool_pairs['token1_symbol'] == p1)]
    pool_address = df_map_pool_pair['pool_address'].values[0]
    
    query = f"""
        SELECT 
        address,
        amount0,
        amount1,
        sender,
        to_address,
        transaction_hash,
        DATE_TRUNC('minute', to_timestamp(block_timestamp)) AS "timestamp"
        FROM eth.{dex}.event_burns
        WHERE address = '{pool_address}'
        ORDER BY "timestamp" DESC
    """
    
    df = client.query(query).read_pandas()
    
    return df

def get_pair_burns_uniswapV3(pool_address, dex="uniswapv3"):
    query = f"""
        SELECT 
        address,
        tick_lower,
        tick_upper,
        amount,
        amount0,
        amount1,
        owner,
        transaction_hash,
        block_number,
        DATE_TRUNC('minute', to_timestamp(block_timestamp)) AS "timestamp"
        FROM eth.{dex}.event_burns
        WHERE address = '{pool_address}'
        ORDER BY "timestamp" DESC
    """
    
    df = client.query(query).read_pandas()
    
    return df

In [ ]:
# df_burns_usdc_weth = get_pair_burns_uniswapV2(df_map_pool_pairs, p0='USDC', p1='WETH', dex='uniswap_v2')
# df_burns_usdc_weth.to_csv('burns_usdc_weth_uniswapV2.csv')
# df_burns_weth_usdt = get_pair_burns_uniswapV2(df_map_pool_pairs, p0='WETH', p1='USDT', dex='uniswap_v2')
# df_burns_weth_usdt.to_csv('burns_weth_usdt_uniswapV2.csv')
# df_burns_usdc_usdt = get_pair_burns_uniswapV2(df_map_pool_pairs, p0='USDC', p1='USDT', dex='uniswap_v2')
# df_burns_usdc_usdt.to_csv('burns_usdc_usdt_uniswapV2.csv')

# 0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8 -> USDC/WETH
# 0x3416cf6c708da44db2624d63ea0aaef7113527c6 -> USDC/USDT
df_burns_usdc_weth_v3 = get_pair_burns_uniswapV3("0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8", dex='uniswap_v3')
df_burns_usdc_weth_v3.to_csv('burns_usdc_weth_uniswapV3.csv')
df_burns_usdc_usdt_v3 = get_pair_burns_uniswapV3("0x3416cf6c708da44db2624d63ea0aaef7113527c6", dex='uniswap_v3')
df_burns_usdc_usdt_v3.to_csv('burns_usdc_usdt_uniswapV3.csv')

### Price

In [ ]:
def get_eth_price_uniswapV2(dex='uniswap_v2'):

    query = f"""
        SELECT *
        FROM prices.eth.{dex}
        ORDER BY "timestamp" DESC
    """
    
    df = client.query(query).read_pandas()
    
    return df

In [ ]:
# df_eth = get_eth_price_uniswapV2()
# df_eth.to_csv('eth_usdc_price.csv')

### Others

In [ ]:
# 0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8 -> USDC/WETH
# 0x3416cf6c708da44db2624d63ea0aaef7113527c6 -> USDC/USDT

In [ ]:
def get_event_collects_uniswapV3(pool_address, dex):
    query = f"""
        SELECT 
        address,
        tick_lower,
        tick_upper,
        amount0,
        amount1,
        owner,
        recipient,
        transaction_hash,
        DATE_TRUNC('minute', to_timestamp(block_timestamp)) AS "timestamp"
        FROM eth.{dex}.event_collects
        WHERE address = '{pool_address}'
        ORDER BY "timestamp" DESC
    """

    df = client.query(query).read_pandas()

    return df

def get_event_flashes_uniswapV3(pool_address, dex):
    query = f"""
        SELECT 
        address,
        amount0,
        amount1,
        paid0,
        paid1,
        sender,
        recipient,
        transaction_hash,
        DATE_TRUNC('minute', to_timestamp(block_timestamp)) AS "timestamp"
        FROM eth.{dex}.event_flashes
        WHERE address = '{pool_address}'
        ORDER BY "timestamp" DESC
    """

    df = client.query(query).read_pandas()

    return df

def get_event_increase_observation_cardinality_nexts_uniswapV3(pool_address, dex):
    query = f"""
        SELECT 
        address,
        observation_cardinality_next_old,
        observation_cardinality_next_new,
        transaction_hash,
        DATE_TRUNC('minute', to_timestamp(block_timestamp)) AS "timestamp"
        FROM eth.{dex}.event_increase_observation_cardinality_nexts
        WHERE address = '{pool_address}'
        ORDER BY "timestamp" DESC
    """

    df = client.query(query).read_pandas()

    return df

dex = "uniswap_v3"

df_event_collects_usdc_weth = get_event_collects_uniswapV3("0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8", dex)
df_event_collects_usdc_usdt = get_event_collects_uniswapV3("0x3416cf6c708da44db2624d63ea0aaef7113527c6", dex)
df_event_collects_usdc_weth.to_csv('event_collects_usdc_weth_uniswapV3.csv')
df_event_collects_usdc_weth.to_csv('event_collects_usdc_usdt_uniswapV3.csv')

df_event_flashes_usdc_weth = get_event_flashes_uniswapV3("0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8", dex)
df_event_flashes_usdc_usdt = get_event_flashes_uniswapV3("0x3416cf6c708da44db2624d63ea0aaef7113527c6", dex)
df_event_flashes_usdc_weth.to_csv('event_flashes_usdc_weth_uniswapV3.csv')
df_event_flashes_usdc_usdt.to_csv('event_flashes_usdc_usdt_uniswapV3.csv')

df_event_increase_observation_cardinality_nexts_usdc_weth = get_event_increase_observation_cardinality_nexts_uniswapV3("0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8", dex)
df_event_increase_observation_cardinality_nexts_usdc_usdt = get_event_increase_observation_cardinality_nexts_uniswapV3("0x3416cf6c708da44db2624d63ea0aaef7113527c6", dex)
df_event_increase_observation_cardinality_nexts_usdc_weth.to_csv('event_increase_observation_cardinality_nexts_usdc_weth_uniswapV3.csv')
df_event_increase_observation_cardinality_nexts_usdc_usdt.to_csv('event_increase_observation_cardinality_nexts_usdc_usdt_uniswapV3.csv')

In [ ]:
query = f"""
DESCRIBE eth.uniswap_v3.event_swaps
"""

client.query(query).read_pandas()